In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from helpers import *
from matplotlib import pyplot as plt
import numpy as np
from experiment_random import *
from scipy.special import expit
import pandas as pd
import seaborn as sns

Using TensorFlow backend.


Initialized TensorFlow


In [2]:
# probability of failure
p = 1e-2

# fixing random seed
np.random.seed(42)

# the random network
exp = RandomExperiment(N = [100, 21, 22, 23, 1], p_inference = [0, p, 0, 0, 0], KLips = 1, activation = 'sigmoid')

# obtaining correct model
model = exp.model_correct

### Testing expression:
$$\sum\limits_{i,j}\frac{\partial y_k}{\partial w_{ij}}w_{ij}=\sum\limits_{j}\frac{\partial y_k}{\partial x_j}x_j
    = -\frac{1}{p}(\mathbb{E}_\xi\Delta_L^k+O(p^2))$$

In [3]:
# obtaining some data point
some_x = -np.random.rand(100) * 100

In [4]:
def val_for_some_x(tensor):
    """ Evaluate tensor for some fixed input """
    return get_session().run(tensor, feed_dict = {model.layers[0].input.name: [some_x]})

In [5]:
def grad_and_sum(output, variable):
    """ Calculate sum(doutput/dvariable * variable) """
    dydvar = tf.gradients(model.layers[-1].output, variable)[0]
    mult = tf.multiply(dydvar, variable)
    return tf.reduce_sum(mult)

In [6]:
def calc_and_print(layer = 0):
    """ Calculate and print quantities from Th1 for a layer """
    dydx = grad_and_sum(model.layers[-1].output, model.layers[layer].input)
    dydw = grad_and_sum(model.layers[-1].output, model.layers[layer].weights[0])
    return(val_for_some_x(dydx), val_for_some_x(dydw))

In [7]:
# comparing two different ways of computing the expression
model.predict(some_x.reshape(1, -1))
for layer in range(len(model.layers)):
    print(layer, calc_and_print(layer))

0 (0.00016751155, 0.00016751171)
1 (0.005007021, 0.005007021)
2 (0.03262172, 0.032621723)
3 (-0.17164078, -0.17164078)


In [8]:
# comparing experimental and theoretical errors
arr_x = np.array([some_x])
np.mean(exp.compute_error(arr_x, repetitions = 1000000))

-5.350309e-05

In [9]:
exp.get_bound_b3(arr_x)['mean'], exp.get_bound_b4(arr_x)['mean']

(array([[-5.007021e-05]], dtype=float32),
 array([[-5.4098815e-05]], dtype=float32))